# Data import

Importa dados do GeoBR para o banco de dados


Imports:

In [56]:
from geobr import read_municipality, read_region, read_state, read_metro_area, read_schools
from sqlalchemy import create_engine
from slugify import slugify
import pandas as pd

import psycopg2

import warnings

# Mute deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Dowload dos dados:

In [57]:
municipalities = read_municipality(code_muni="all")
regions = read_region()
states = read_state()
metro_area = read_metro_area()
schools = read_schools()

Filtros e transformações:

In [58]:
municipalities = municipalities[["code_muni", "name_muni", "code_state"]]
regions = regions[["code_region", "name_region"]]
states = states[["code_state", "abbrev_state", "name_state", "code_region"]]
metro_area = metro_area[["code_muni", "code_state", "name_metro", "type", "subdivision", "legislation", "legislation_date"]]
schools = schools[["code_school", "abbrev_state", "name_muni", "name_school", "education_level", "education_level_others", "address", "phone_number", "government_level", "private_school_type", "service_restriction", "size", "location_type", "urban", "date_update"]]

In [59]:
# Ajustando formato de data
metro_area["legislation_date"] = metro_area["legislation_date"].apply(lambda date_string: "-".join(date_string.split(".")[::-1]))

# Convertendo campos pra integer
metro_area = metro_area.astype({"code_muni": "int", "code_state": "int"})

# remove as linhas com datas invalidas
metro_area = metro_area[metro_area.legislation_date.str.match(r"(\d+)\-(\d+)\-(\d+)")]

# converte o campo legislation_date para date
metro_area["legislation_date"] = pd.to_datetime(metro_area["legislation_date"]).dt.date


In [60]:
# convertendo o campo date_update para date
schools["date_update"] = pd.to_datetime(schools["date_update"]).dt.date

In [61]:
# criando dict {"estado": "codigo"}
state_code_list = {}

for index, state in states.iterrows():
  state_code_list[state["abbrev_state"]] = state["code_state"]

# atribuindo codigo do estado
schools["code_state"] = schools.apply(lambda school: int(state_code_list[school["abbrev_state"]]), axis=1)

In [62]:
schools["city_slug"] = schools.apply(lambda school: slugify(school["name_muni"]), axis=1)
municipalities["city_slug"] = municipalities.apply(lambda city: slugify(city["name_muni"]), axis=1)

In [63]:
from collections import defaultdict

unknown_cities = defaultdict(list)

for index, school in schools.iterrows():
  school_city_slug = school["city_slug"]
  existing_muni = municipalities[municipalities.city_slug == school_city_slug].head(1)

  if existing_muni.empty:
    unknown_cities[school_city_slug].append(school["code_school"])


import json
print(json.dumps(unknown_cities,indent=4))

{
    "mojui-dos-campos": [
        15012026,
        15012107,
        15012131,
        15012158,
        15012220,
        15012271,
        15012409,
        15012590,
        15012620,
        15012638,
        15012743,
        15013219,
        15013367,
        15013502,
        15013634,
        15013669,
        15013707,
        15013731,
        15013782,
        15013928,
        15014053,
        15014061,
        15014096,
        15014142,
        15014223,
        15014444,
        15014614,
        15014630,
        15014649,
        15014690,
        15014720,
        15014800,
        15014819,
        15014991,
        15015068,
        15015076,
        15016668,
        15016676,
        15016684,
        15016692,
        15147622,
        15151557,
        15167399,
        15168328,
        15222411,
        15222608,
        15518647,
        15518698,
        15519244,
        15538540,
        15551482,
        15551520,
        15572579,
        15572609,


In [64]:
# with open("unknown_cities.json", "w") as file:
#   json.dump(unknown_cities, file, indent=4)

print(len(list(unknown_cities.keys())))

19


In [65]:
# criando dict {"estado": "codigo"}
muni_code_list = {}

for index, muni in municipalities.iterrows():
  muni_code_list[slugify(muni["name_muni"])] = muni["code_muni"]
muni_code_list[slugify("mojui-dos-campos")] = 5300109
muni_code_list[slugify("serra-caiada")] = 5300110
muni_code_list[slugify("joca-claudino")] = 5300111
muni_code_list[slugify("sao-vicente-do-serido")] = 5300112
muni_code_list[slugify("brazopolis")] = 5300113
muni_code_list[slugify("embu-das-artes")] = 5300114
muni_code_list[slugify("balneario-rincao")] = 5300115
muni_code_list[slugify("pescaria-brava")] = 5300116
muni_code_list[slugify("pinto-bandeira")] = 5300117
muni_code_list[slugify("paraiso-das-aguas")] = 5300118


muni_code_list[slugify("santa-izabel-do-para")] = 1506500
muni_code_list[slugify("eldorado-do-carajas")] = 1502954
muni_code_list[slugify("itapaje")] = 2306306
muni_code_list[slugify("iguaracy")] = 2606903
muni_code_list[slugify("muquem-do-sao-francisco")] = 2922250
muni_code_list[slugify("mogi-mirim")] = 3530805
muni_code_list[slugify("florinea")] = 3516101
muni_code_list[slugify("sao-luiz-do-paraitinga")] = 3550001
muni_code_list[slugify("poxoreu")] = 5107008


#muni_code_list

# atribuindo codigo do estado
schools["code_muni"] = schools.apply(lambda school1: int(muni_code_list[slugify(school1["name_muni"])]), axis=1)

## Dump dos dados

Declarando funções de dump:

In [66]:
def create_table_from_dataframe(df, table_name, engine):
    df.head(0).to_sql(table_name, engine, if_exists='replace',index=False)
 
def populate_table_with_dataframe(df, table, connection):
        # Create a list of tupples from the dataframe values
        tuples = [tuple(x) for x in df.to_numpy()]
    
        # Comma-separated dataframe columns
        column_list = list(df.columns)
        cols = ','.join(column_list)
        
        value_placeholders = ",".join(["%s" for _ in range(len(column_list))])
        # SQL query to execute
        query = f"INSERT INTO {table}({cols}) VALUES({value_placeholders})"
        try:
            cursor = connection.cursor()
            cursor.executemany(query, tuples)
            connection.commit()

        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            connection.rollback()
            return 1

Declarando parâmetros e iniciando conexão:

In [67]:
pg_user = "postgres"
pg_pass = "password"
pg_host = "localhost"
pg_port = "5432"
pg_db = "postgres"

engine = create_engine(
    f"postgresql+psycopg2://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}"
)
conn = engine.raw_connection()

Escrever dataframes no DB:

In [68]:
# Regions
create_table_from_dataframe(regions, "regions", engine)
populate_table_with_dataframe(regions, "regions", conn)

# States
create_table_from_dataframe(states, "states", engine)
populate_table_with_dataframe(states, "states", conn)

# Municipalities
create_table_from_dataframe(municipalities, "municipalities", engine)
populate_table_with_dataframe(municipalities, "municipalities", conn)

# Metro Area
create_table_from_dataframe(metro_area, "metro_area", engine)
populate_table_with_dataframe(metro_area, "metro_area", conn)

# schools
create_table_from_dataframe(schools, "schools", engine)
populate_table_with_dataframe(schools, "schools", conn)